# Imports and constants:

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
assert os.environ["CUDA_VISIBLE_DEVICES"] == "1"

import hydra
from omegaconf import DictConfig, OmegaConf
import argparse
import tqdm
from pprint import pprint
from immunization_utils_dev import *
from tqdm import tqdm, trange

# Reload (Dev)

In [ ]:
from importlib import reload
import importlib
reload(importlib.import_module('immunization_utils_dev'))
from immunization_utils_dev import *


# Initialize:

In [ ]:
cfg = OmegaConf.load('config/base.yaml')
cfg.override='debug'
config_overrides = OmegaConf.load(f'config/overrides/{cfg.override}.yaml')
cfg = OmegaConf.merge(cfg, config_overrides)
kwargs, \
    logging_dict, \
    model, tokenizer, \
    eval_model, eval_tokenizer, \
    training_attack_data_dict, \
    safety_eval_data, \
    performance_eval_data = initialize(cfg)

# Go Play

In [ ]:
layer=15
kwargs['init_defence_epochs'] = 150
kwargs['init_attack_prompts'] = 30
kwargs['init_attack_epochs'] = 30
kwargs['init_defence_prompts'] = 500
kwargs['defence_reg_coeff'] = 0.02
kwargs['defence_strategy'] = 'GATE_UP_DOWN_QUERY_KEY_VALUE_OUTPUT'  
# kwargs['defence_strategy'] = 'GATE_UP_DOWN'
kwargs['init_defence_criterion'] = 'mse'
kwargs['frobenious_norm_scaling_factor'] = 0.7
kwargs['cosine_similarity_scaling_factor'] = 0.3
kwargs['init_eval_safety_prompts'] = 50
kwargs['performance_batches'] = 30
kwargs['defence_regularization'] = 'simple'

In [ ]:
# Initialize a configuration for attack
attack_config = init_single_layer_attack_config(model, layer, kwargs)

attacked_model, safety_eval_table = reft_attack(
                    model, 
                    tokenizer, 
                    attack_config, 
                    training_attack_data_dict, 
                    eval_model, 
                    eval_tokenizer, 
                    safety_eval_data,
                    performance_eval_data,
                    logging_dict, 
                    kwargs)

In [ ]:
print('Toxicity: ',attack_config['toxicity'])

In [ ]:
# Initialize a configuration for defence:
defence_config = init_custom_defence_config(model, attack_config, attacked_model, 1, kwargs)     
kwargs['first_inner_defence_round']= True

In [ ]:

eval_table, defence_results = custom_defence(
                model,
                tokenizer,
                eval_model,
                eval_tokenizer,
                defence_config,
                training_attack_data_dict,
                safety_eval_data,
                performance_eval_data,
                logging_dict, 
                kwargs)

kwargs['first_inner_defence_round']= False

In [ ]:
model = reset_defended_module(model, defence_config, kwargs)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 8))  # Set the figure size to make the plot bigger

# smooth the series:
def smooth_series(series, smoothing_factor=0.9):
    smoothed_series = []
    last_value = series[0]
    for value in series:
        smoothed_value = last_value * smoothing_factor + value * (1 - smoothing_factor)
        smoothed_series.append(smoothed_value)
        last_value = smoothed_value
    return smoothed_series


epoch_mlp_reg_losses = [r.detach().cpu().double() for r in defence_results['epoch_mlp_reg_losses']]
epoch_mlp_def_losses = [r.detach().cpu().double() for r in defence_results['epoch_mlp_def_losses']]
epoch_attn_def_losses = [r.detach().cpu().double() for r in defence_results['epoch_attn_def_losses']]
epoch_attn_reg_losses = [r.detach().cpu().double() for r in defence_results['epoch_attn_reg_losses']]
epoch_block_def_losses = [r.detach().cpu().double() for r in defence_results['epoch_block_def_losses']]
epoch_block_reg_losses = [r.detach().cpu().double() for r in defence_results['epoch_block_reg_losses']]
# smoothed_main_losses_series = smooth_series(main_losses_series)
# plt.plot(smoothed_main_losses_series, label='smoothed_epoch_mlp_reg_losses')

plt.plot(epoch_block_def_losses, label='epoch_block_def_losses', alpha=0.5, linestyle='-.')
#plt.plot(epoch_block_reg_losses, label='epoch_block_reg_losses', alpha=0.5)
#plt.plot(epoch_mlp_reg_losses, label='epoch_mlp_reg_losses', alpha=0.5)
plt.plot(epoch_mlp_def_losses, label='epoch_mlp_def_losses', alpha=0.5)
plt.plot(epoch_attn_def_losses, label='epoch_attn_def_losses', alpha=0.7, linestyle='--')
#plt.plot(epoch_attn_reg_losses, label='epoch_attn_reg_losses', alpha=0.5)
#plt.plot([r.cpu().double() for r in defence_results['epoch_reg_losses']], label='reg_loss')
plt.yscale('log')
plt.legend()
plt.show()

In [ ]:
epoch_block_def_losses = [r.detach().cpu().double() for r in defence_results['epoch_block_def_losses']]
smoothed_epoch_block_def_losses = smooth_series(epoch_block_def_losses)
plt.plot(smoothed_epoch_block_def_losses, label='smoothed_epoch_block_def_losses')
plt.plot(epoch_block_def_losses, label='epoch_block_def_losses', alpha=0.5)
#plt.plot([r.cpu().double() for r in defence_results['epoch_reg_losses']], label='reg_loss')
plt.legend()
plt.show()

In [ ]:
epoch_block_def_losses = [r.detach().cpu().double() for r in defence_results['epoch_block_def_losses']]
smoothed_epoch_block_def_losses = smooth_series(epoch_block_def_losses)
plt.plot(smoothed_epoch_block_def_losses, label='smoothed_epoch_block_def_losses')
plt.plot(epoch_block_def_losses, label='epoch_block_def_losses', alpha=0.5)
#plt.plot([r.cpu().double() for r in defence_results['epoch_reg_losses']], label='reg_loss')
plt.legend()
plt.show()

In [ ]:
epoch_block_def_losses = [r.detach().cpu().double() for r in defence_results['epoch_block_def_losses']]
smoothed_epoch_block_def_losses = smooth_series(epoch_block_def_losses)
plt.plot(smoothed_epoch_block_def_losses, label='smoothed_epoch_block_def_losses')
plt.plot(epoch_block_def_losses, label='epoch_block_def_losses', alpha=0.5)
#plt.plot([r.cpu().double() for r in defence_results['epoch_reg_losses']], label='reg_loss')
plt.legend()
plt.show()

In [ ]:
epoch_block_reg_losses = [r.detach().cpu().double() for r in defence_results['epoch_block_reg_losses']]
smoothed_epoch_block_reg_losses = smooth_series(epoch_block_reg_losses)
plt.plot(smoothed_epoch_block_reg_losses, label='smoothed_epoch_block_reg_losses')
plt.plot(epoch_block_reg_losses, label='epoch_block_reg_losses', alpha=0.5)
plt.legend()
plt.show()

In [ ]:
epoch_mlp_def_losses = [r.detach().cpu().double() for r in defence_results['epoch_mlp_def_losses']]
smoothed_mlp_def_losses_series = smooth_series(epoch_mlp_def_losses)
plt.plot(smoothed_mlp_def_losses_series, label='smoothed_epoch_mlp_def_losses')
plt.plot(epoch_mlp_def_losses, label='epoch_mlp_def_losses', alpha=0.5)
#plt.plot([r.cpu().double() for r in defence_results['epoch_reg_losses']], label='reg_loss')
plt.legend()
plt.show()

In [ ]:
epoch_mlp_def_losses = [r.detach().cpu().double() for r in defence_results['epoch_mlp_def_losses']]
smoothed_mlp_def_losses_series = smooth_series(epoch_mlp_def_losses)
plt.plot(smoothed_mlp_def_losses_series, label='smoothed_epoch_mlp_def_losses')
plt.plot(epoch_mlp_def_losses, label='epoch_mlp_def_losses', alpha=0.5)
#plt.plot([r.cpu().double() for r in defence_results['epoch_reg_losses']], label='reg_loss')
plt.legend()
plt.show()